<a href="https://colab.research.google.com/github/chayakim/EFIMM0142_Individual-assignment_2549163/blob/main/Modelling_Individual_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Question 1A**

In [1]:
# Install pulp library
!pip install pulp

# Import PuLP library for linear and integer programming
from pulp import *
import pandas as pd

# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': 30, 'C': 10}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Total_Cost"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 150]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nTotal Cost:", value(prob.objective))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 16.0 MB/s eta 0:00:00

Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B        90.0        0.0
6    Mar       B       100.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       162.0       42.0
9    Feb       C       201.0      113.0
10   Mar       C       126.0       99.0
11   Apr       C        51.0        0.0

Total Cost: 26127.0


# **Question 1B**

In [ ]:
# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': {'Jan': 30, 'Feb': 30, 'Mar': 35, 'Apr': 30}, 'C': 10} # Change from 30 to 35
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p][m] * x[f"{p}_{m}"] if isinstance(production_cost[p], dict) else production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Total_Cost"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 150]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nTotal Cost:", value(prob.objective))



Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B       190.0      100.0
6    Mar       B         0.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       179.0       59.0
9    Feb       C        71.0        0.0
10   Mar       C       239.0       99.0
11   Apr       C        51.0        0.0

Total Cost: 26279.0


# **Question 1C**

In [ ]:
# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': 30, 'C': 10}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Total_Cost"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 170]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nTotal Cost:", value(prob.objective))


Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B        90.0        0.0
6    Mar       B       100.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       182.0       62.0
9    Feb       C       201.0      133.0
10   Mar       C       126.0      119.0
11   Apr       C        51.0        0.0

Total Cost: 26357.0


# **Question 1D**

In [ ]:
# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': 30, 'C': 10}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Total_Cost"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 150]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )
# Add total inventory holding cost constraint
total_holding_cost_limit = 500  # Maximum allowable holding cost
prob += (
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months) <= total_holding_cost_limit,
    "Total_Holding_Cost_Constraint"
)
# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nTotal Cost:", value(prob.objective))


Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B        90.0        0.0
6    Mar       B       100.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       162.0       42.0
9    Feb       C       201.0      113.0
10   Mar       C       126.0       99.0
11   Apr       C        51.0        0.0

Total Cost: 26127.0


# **Question 2A**

In [19]:
# Install pulp library
!pip install pulp

# Importing the library
from pulp import *

# Data
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
             "Los Angeles", "Glendale", "Jacksonville", "Little Rock",
             "Bridgeport", "Sacramento"]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Production costs
production_cost = {"Cincinnati": 1.20, "Oakland": 1.65}

# Freight costs
freight_cost = {
    "Cincinnati": [10000, 0.84, 0.83, 0.45, 0.36, 10000, 10000, 0.34, 0.34, 0.34, 10000],
    "Oakland": [0.22, 0.74, 0.49, 10000, 10000, 0.22, 0.22, 10000, 10000, 10000, 0.15]
}

# Capacities
max_cincinnati = 500000
max_oakland = 500000

# Initialize problem
model = LpProblem("Minimize_Total_Cost", LpMinimize)

# Decision variables
x = {
    (facility, location): LpVariable(f"x_{facility}_{location}", lowBound=0, cat="Continuous")
    for facility in ["Cincinnati", "Oakland"]
    for location in locations
}

# Objective function: Minimize total production and shipping cost
model += lpSum(
    (production_cost[facility] + freight_cost[facility][i]) * x[(facility, location)]
    for facility in ["Cincinnati", "Oakland"]
    for i, location in enumerate(locations)
)

# Constraints: Demand must be satisfied for each location
for i, location in enumerate(locations):
    model += lpSum(x[(facility, location)] for facility in ["Cincinnati", "Oakland"]) == demand[i]

# Capacity constraint for Cincinnati
model += lpSum(x[("Cincinnati", location)] for location in locations) <= max_cincinnati

# Capacity constraint for Oakland
model += lpSum(x[("Oakland", location)] for location in locations) <= max_oakland

# Solve the problem
model.solve()

# Extract results including allocations from both facilities to each location
results = []
total_cost = 0
for location in locations:
    allocation_cincinnati = x[("Cincinnati", location)].varValue if x[("Cincinnati", location)].varValue > 0 else 0
    allocation_oakland = x[("Oakland", location)].varValue if x[("Oakland", location)].varValue > 0 else 0
    cost_cincinnati = (
        allocation_cincinnati
        * (production_cost["Cincinnati"] + freight_cost["Cincinnati"][locations.index(location)])
    )
    cost_oakland = (
        allocation_oakland
        * (production_cost["Oakland"] + freight_cost["Oakland"][locations.index(location)])
    )
    location_cost = cost_cincinnati + cost_oakland
    total_cost += location_cost

    results.append({
        "Location": location,
        "From Cincinnati": allocation_cincinnati,
        "From Oakland": allocation_oakland,
        "Total Gallons": allocation_cincinnati + allocation_oakland,
        "Cost": location_cost
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df.loc["Total"] = ["-", results_df["From Cincinnati"].sum(), results_df["From Oakland"].sum(), results_df["Total Gallons"].sum(), total_cost]

# Display results
print("Results:")
print(results_df)

Results:
           Location  From Cincinnati  From Oakland  Total Gallons        Cost
0         Santa Ana              0.0       22418.0        22418.0    41921.66
1           El Paso           6800.0           0.0         6800.0    13872.00
2         Pendleton          39636.0       40654.0        80290.0   167460.64
3           Houston         100447.0           0.0       100447.0   165737.55
4       Kansas City          24570.0           0.0        24570.0    38329.20
5       Los Angeles              0.0       64761.0        64761.0   121103.07
6          Glendale              0.0       33689.0        33689.0    62998.43
7      Jacksonville          68486.0           0.0        68486.0   105468.44
8       Little Rock         148586.0           0.0       148586.0   228822.44
9        Bridgeport         111475.0           0.0       111475.0   171671.50
10       Sacramento              0.0      112000.0       112000.0   201600.00
Total             -         500000.0      273522.0     

# **Question 2B & 2C**

In [15]:
# Calculate Total Cost
total_cost = 0
for i, location in enumerate(locations):
    allocation_cincinnati = x[("Cincinnati", location)].varValue or 0
    allocation_oakland = x[("Oakland", location)].varValue or 0
    cost_cincinnati = allocation_cincinnati * (production_cost["Cincinnati"] + freight_cost["Cincinnati"][i])
    cost_oakland = allocation_oakland * (production_cost["Oakland"] + freight_cost["Oakland"][i])
    total_cost += cost_cincinnati + cost_oakland

# Calculate total demand
total_demand = sum(demand) # Calculating and defining the total demand

# Breakeven Point
breakeven_price = total_cost / total_demand


# Output Results
print(f"Total Cost: ${total_cost:,.2f}")
print(f"Total Demand: {total_demand} gallons")
print(f"Break-even Price: ${breakeven_price:.2f} per gallon")

Total Cost: $1,318,984.93
Total Demand: 773522 gallons
Break-even Price: $1.71 per gallon


In [16]:
# Calculate Total Cost
total_cost = 0
for i, location in enumerate(locations):
    allocation_cincinnati = x[("Cincinnati", location)].varValue or 0
    allocation_oakland = x[("Oakland", location)].varValue or 0
    cost_cincinnati = allocation_cincinnati * (production_cost["Cincinnati"] + freight_cost["Cincinnati"][i])
    cost_oakland = allocation_oakland * (production_cost["Oakland"] + freight_cost["Oakland"][i])
    total_cost += cost_cincinnati + cost_oakland

# Calculate total demand
total_demand = sum(demand) # Calculating and defining the total demand

# Breakeven Point
breakeven_price = total_cost / total_demand

#Calculate the bid price with a 15% markup
markup_percentage = 0.15
bid_price = breakeven_price * (1 + markup_percentage)


# Output Results
print(f"Total Cost: ${total_cost:,.2f}")
print(f"Total Demand: {total_demand} gallons")
print(f"Break-even Price: ${breakeven_price:.2f} per gallon")
print(f"Bid Price with 15% Markup: ${bid_price:.2f} per gallon")

Total Cost: $1,318,984.93
Total Demand: 773522 gallons
Break-even Price: $1.71 per gallon
Bid Price with 15% Markup: $1.96 per gallon


# **Question 2D**

In [17]:
# Data
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
             "Los Angeles", "Glendale", "Jacksonville", "Little Rock",
             "Bridgeport", "Sacramento"]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Production costs
production_cost = {"Cincinnati": 1.20, "Oakland": 1.65}

# Freight costs
freight_cost = {
    "Cincinnati": [10000, 0.84, 0.83, 0.45, 0.36, 10000, 10000, 0.34, 0.34, 0.34, 10000],
    "Oakland": [0.22, 0.74, 0.49, 10000, 10000, 0.22, 0.22, 10000, 10000, 10000, 0.15]
}

# Capacities
max_cincinnati = 500000

# Initialize problem
model = LpProblem("Minimize_Total_Cost", LpMinimize)

# Decision variables
x = {
    (facility, location): LpVariable(f"x_{facility}_{location}", lowBound=0, cat="Continuous")
    for facility in ["Cincinnati", "Oakland"]
    for location in locations
}

# Objective function: Minimize total production and shipping cost
model += lpSum(
    (production_cost[facility] + freight_cost[facility][i]) * x[(facility, location)]
    for facility in ["Cincinnati", "Oakland"]
    for i, location in enumerate(locations)
)

# Constraints: Demand must be satisfied for each location
for i, location in enumerate(locations):
    model += lpSum(x[(facility, location)] for facility in ["Cincinnati", "Oakland"]) == demand[i]

# Capacity constraint for Cincinnati
model += lpSum(x[("Cincinnati", location)] for location in locations) <= max_cincinnati

# Solve the problem
model.solve()

# Extract results including allocations from both facilities to each location
results = []
total_cost = 0
for location in locations:
    allocation_cincinnati = x[("Cincinnati", location)].varValue if x[("Cincinnati", location)].varValue > 0 else 0
    allocation_oakland = x[("Oakland", location)].varValue if x[("Oakland", location)].varValue > 0 else 0
    cost_cincinnati = (
        allocation_cincinnati
        * (production_cost["Cincinnati"] + freight_cost["Cincinnati"][locations.index(location)])
    )
    cost_oakland = (
        allocation_oakland
        * (production_cost["Oakland"] + freight_cost["Oakland"][locations.index(location)])
    )
    location_cost = cost_cincinnati + cost_oakland
    total_cost += location_cost

    results.append({
        "Location": location,
        "From Cincinnati": allocation_cincinnati,
        "From Oakland": allocation_oakland,
        "Total Gallons": allocation_cincinnati + allocation_oakland,
        "Cost": location_cost
    })


# Define fluctuation scenarios for freight costs (-10%, 0%, +10%)
freight_cost_fluctuations = [-0.10, 0.00, 0.10]

# Initialize results list
fluctuation_results = []

for fluctuation in freight_cost_fluctuations:
    adjusted_total_cost = 0

    # Adjust freight costs for fluctuation
    adjusted_freight_cost = {
        "Cincinnati": [cost * (1 + fluctuation) if cost < 10000 else cost for cost in freight_cost["Cincinnati"]],
        "Oakland": [cost * (1 + fluctuation) if cost < 10000 else cost for cost in freight_cost["Oakland"]]
    }

    # Recalculate total cost with adjusted freight costs
    for location in locations:
        allocation_cincinnati = x[("Cincinnati", location)].varValue if x[("Cincinnati", location)].varValue > 0 else 0
        allocation_oakland = x[("Oakland", location)].varValue if x[("Oakland", location)].varValue > 0 else 0

        cost_cincinnati = (
            allocation_cincinnati * (production_cost["Cincinnati"] + adjusted_freight_cost["Cincinnati"][locations.index(location)])
        )
        cost_oakland = (
            allocation_oakland * (production_cost["Oakland"] + adjusted_freight_cost["Oakland"][locations.index(location)])
        )
        adjusted_total_cost += cost_cincinnati + cost_oakland

    # Append results for each scenario
    fluctuation_results.append({
        "Fluctuation (%)": f"{fluctuation * 100:.0f}%",
        "Adjusted Total Cost": adjusted_total_cost,
        "Average Cost per Gallon": adjusted_total_cost / sum(demand)
    })

# Convert results to a DataFrame
fluctuation_results_df = pd.DataFrame(fluctuation_results)

# Print and display the results
print("Freight Cost Fluctuation Analysis for a Two-Year Contract:")
print(fluctuation_results_df)

Freight Cost Fluctuation Analysis for a Two-Year Contract:
  Fluctuation (%)  Adjusted Total Cost  Average Cost per Gallon
0            -10%          1292217.567                 1.670563
1              0%          1318984.930                 1.705168
2             10%          1345752.293                 1.739772


# **Question 2E**

In [8]:
# Importing the library
from pulp import *

# Data
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
             "Los Angeles", "Glendale", "Jacksonville", "Little Rock",
             "Bridgeport", "Sacramento"]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Production costs
production_cost = {"Cincinnati": 1.20, "Oakland": 1.65}

# Freight costs
freight_cost = {
    "Cincinnati": [10000, 0.84, 0.83, 0.45, 0.36, 10000, 10000, 0.34, 0.34, 0.34, 10000],
    "Oakland": [0.22, 0.74, 0.49, 10000, 10000, 0.22, 0.22, 10000, 10000, 10000, 0.15]
}

# Capacities
max_cincinnati = 500000
max_oakland = 500000

def solve_problem(production_costs, freight_costs):
    # Initialize problem
    model = LpProblem("Minimize_Total_Cost", LpMinimize)

    # Decision variables
    x = {
        (facility, location): LpVariable(f"x_{facility}_{location}", lowBound=0, cat="Continuous")
        for facility in ["Cincinnati", "Oakland"]
        for location in locations
    }

    # Objective function: Minimize total production and shipping cost
    model += lpSum(
        (production_costs[facility] + freight_costs[facility][i]) * x[(facility, location)]
        for facility in ["Cincinnati", "Oakland"]
        for i, location in enumerate(locations)
    )

    # Constraints: Demand must be satisfied for each location
    for i, location in enumerate(locations):
        model += lpSum(x[(facility, location)] for facility in ["Cincinnati", "Oakland"]) == demand[i]

    # Capacity constraint for Cincinnati
    model += lpSum(x[("Cincinnati", location)] for location in locations) <= max_cincinnati

    # Capacity constraint for Oakland
    model += lpSum(x[("Oakland", location)] for location in locations) <= max_oakland

    # Solve the problem
    model.solve()

    # Extract results including allocations from both facilities to each location
    results = []
    total_cost = 0
    for location in locations:
        allocation_cincinnati = x[("Cincinnati", location)].varValue if x[("Cincinnati", location)].varValue > 0 else 0
        allocation_oakland = x[("Oakland", location)].varValue if x[("Oakland", location)].varValue > 0 else 0
        cost_cincinnati = (
            allocation_cincinnati
            * (production_costs["Cincinnati"] + freight_costs["Cincinnati"][locations.index(location)])
        )
        cost_oakland = (
            allocation_oakland
            * (production_costs["Oakland"] + freight_costs["Oakland"][locations.index(location)])
        )
        location_cost = cost_cincinnati + cost_oakland
        total_cost += location_cost

        results.append({
            "Location": location,
            "From Cincinnati": allocation_cincinnati,
            "From Oakland": allocation_oakland,
            "Total Gallons": allocation_cincinnati + allocation_oakland,
            "Cost": location_cost
        })

    return results, total_cost

# Base Scenario: Original costs
results_base_scenario, total_cost_base_scenario = solve_problem(production_cost, freight_cost)

# Scenario 1: Increase of Production costs in Cincinnati by 10%
production_cost_scenario_1 = production_cost.copy()
production_cost_scenario_1["Cincinnati"] *= 1.10

results_scenario_1, total_cost_scenario_1 = solve_problem(production_cost_scenario_1, freight_cost)

# Scenario 2: Increase of Production costs in Oakland by 10%
production_cost_scenario_2 = production_cost.copy()
production_cost_scenario_2["Oakland"] *= 1.10

results_scenario_2, total_cost_scenario_2 = solve_problem(production_cost_scenario_2, freight_cost)

# Scenario 3: Increase of Freight costs in Cincinnati by 10%
freight_cost_scenario_3 = freight_cost.copy()
freight_cost_scenario_3["Cincinnati"] = [cost * 1.10 for cost in freight_cost["Cincinnati"]]

results_scenario_3, total_cost_scenario_3 = solve_problem(production_cost, freight_cost_scenario_3)

# Scenario 4: Increase of Freight costs in Oakland by 10%
freight_cost_scenario_4 = freight_cost.copy()
freight_cost_scenario_4["Oakland"] = [cost * 1.10 for cost in freight_cost["Oakland"]]

results_scenario_4, total_cost_scenario_4 = solve_problem(production_cost, freight_cost_scenario_4)

# Scenario 5: Decrease of Production costs in Cincinnati by 10%
production_cost_scenario_5 = production_cost.copy()
production_cost_scenario_5["Cincinnati"] *= 0.90

results_scenario_5, total_cost_scenario_5 = solve_problem(production_cost_scenario_5, freight_cost)

# Scenario 6: Decrease of Production costs in Oakland by 10%
production_cost_scenario_6 = production_cost.copy()
production_cost_scenario_6["Oakland"] *= 0.90

results_scenario_6, total_cost_scenario_6 = solve_problem(production_cost_scenario_6, freight_cost)

# Scenario 7: Decrease of Freight costs in Cincinnati by 10%
freight_cost_scenario_7 = freight_cost.copy()
freight_cost_scenario_7["Cincinnati"] = [cost * 0.90 for cost in freight_cost["Cincinnati"]]

results_scenario_7, total_cost_scenario_7 = solve_problem(production_cost, freight_cost_scenario_7)

# Scenario 8: Decrease of Freight costs in Oakland by 10%
freight_cost_scenario_8 = freight_cost.copy()
freight_cost_scenario_8["Oakland"] = [cost * 0.90 for cost in freight_cost["Oakland"]]

results_scenario_8, total_cost_scenario_8 = solve_problem(production_cost, freight_cost_scenario_8)

# Display results for each scenario
import pandas as pd

def display_results(results):
    results_df = pd.DataFrame(results)
    results_df.loc["Total"] = ["-", results_df["From Cincinnati"].sum(), results_df["From Oakland"].sum(), results_df["Total Gallons"].sum(), results_df["Cost"].sum()]
    return results_df

print("Base Scenario: Original costs")
print(display_results(results_base_scenario))
print(f"Total Cost: {total_cost_base_scenario}\n")

print("Scenario 1: Increase of Production costs in Cincinnati by 10%")
print(display_results(results_scenario_1))
print(f"Total Cost: {total_cost_scenario_1}\n")

print("Scenario 2: Increase of Production costs in Oakland by 10%")
print(display_results(results_scenario_2))
print(f"Total Cost: {total_cost_scenario_2}\n")

print("Scenario 3: Increase of Freight costs in Cincinnati by 10%")
print(display_results(results_scenario_3))
print(f"Total Cost: {total_cost_scenario_3}\n")

print("Scenario 4: Increase of Freight costs in Oakland by 10%")
print(display_results(results_scenario_4))
print(f"Total Cost: {total_cost_scenario_4}\n")

print("Scenario 5: Decrease of Production costs in Cincinnati by 10%")
print(display_results(results_scenario_5))
print(f"Total Cost: {total_cost_scenario_5}\n")

print("Scenario 6: Decrease of Production costs in Oakland by 10%")
print(display_results(results_scenario_6))
print(f"Total Cost: {total_cost_scenario_6}\n")

print("Scenario 7: Decrease of Freight costs in Cincinnati by 10%")
print(display_results(results_scenario_7))
print(f"Total Cost: {total_cost_scenario_7}\n")

print("Scenario 8: Decrease of Freight costs in Oakland by 10%")
print(display_results(results_scenario_8))
print(f"Total Cost: {total_cost_scenario_8}\n")

Base Scenario: Original costs
           Location  From Cincinnati  From Oakland  Total Gallons        Cost
0         Santa Ana              0.0       22418.0        22418.0    41921.66
1           El Paso           6800.0           0.0         6800.0    13872.00
2         Pendleton          39636.0       40654.0        80290.0   167460.64
3           Houston         100447.0           0.0       100447.0   165737.55
4       Kansas City          24570.0           0.0        24570.0    38329.20
5       Los Angeles              0.0       64761.0        64761.0   121103.07
6          Glendale              0.0       33689.0        33689.0    62998.43
7      Jacksonville          68486.0           0.0        68486.0   105468.44
8       Little Rock         148586.0           0.0       148586.0   228822.44
9        Bridgeport         111475.0           0.0       111475.0   171671.50
10       Sacramento              0.0      112000.0       112000.0   201600.00
Total             -         500000

# **Question 3A**

In [6]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum

# Create the problem
prob = LpProblem("PreemptiveGoalProgramming", LpMinimize)

# Define decision variables
G = LpVariable("GasTax", lowBound=0)  # Gas tax per gallon (cents)
T1 = LpVariable("TaxRateUpTo30K", lowBound=0)  # Tax rate on income up to $30,000
T2 = LpVariable("TaxRateAbove30K", lowBound=0)  # Tax rate on income over $30,000
C = LpVariable("SpendingCut", lowBound=0)  # Spending cut (in billions)

# Deviation variables (for each goal)
d1_plus = LpVariable("d1_plus", lowBound=0)  # Positive deviation for Goal 1
d1_minus = LpVariable("d1_minus", lowBound=0)  # Negative deviation for Goal 1
d2_plus = LpVariable("d2_plus", lowBound=0)  # Positive deviation for Goal 2
d2_minus = LpVariable("d2_minus", lowBound=0)  # Negative deviation for Goal 2
d3_plus = LpVariable("d3_plus", lowBound=0)  # Positive deviation for Goal 3
d3_minus = LpVariable("d3_minus", lowBound=0)  # Negative deviation for Goal 3
d4_plus = LpVariable("d4_plus", lowBound=0)  # Positive deviation for Goal 4
d4_minus = LpVariable("d4_minus", lowBound=0)  # Negative deviation for Goal 4

# Constraints
# Revenue constraints from table
revenue_low = G + 20 * T1
revenue_high = 0.5 * G + 5 * T1 + 15 * T2
total_revenue = revenue_low + revenue_high

# Objective function (minimize deviations in priority order)
prob += d1_minus + d2_plus + d3_plus + d4_plus

# Goal 1: Balance the budget (revenues >= costs)
prob += total_revenue + C - 1000 + d1_minus - d1_plus == 0

# Goal 2: Cut spending by at most $150 billion
prob += C - 150 + d2_minus - d2_plus == 0

# Goal 3: Raise at most $550 billion in taxes from the upper class
prob += revenue_high - 550 + d3_minus - d3_plus == 0

# Goal 4: Raise at most $350 billion in taxes from the lower class
prob += revenue_low - 350 + d4_minus - d4_plus == 0

# Fair Tax Policy
prob += T2 >= T1

# Solve preemptively for each priority goal
# Step 1: Minimize d1_minus (highest priority goal)
prob.setObjective(d1_minus)
prob.solve()

# Store results and lock d1_minus
opt_d1_minus = d1_minus.varValue
prob += d1_minus == opt_d1_minus

# Step 2: Minimize d2_plus (second priority goal)
prob.setObjective(d2_plus)
prob.solve()

# Store results and lock d2_plus
opt_d2_plus = d2_plus.varValue
prob += d2_plus == opt_d2_plus

# Step 3: Minimize d3_plus (third priority goal)
prob.setObjective(d3_plus)
prob.solve()

# Store results and lock d3_plus
opt_d3_plus = d3_plus.varValue
prob += d3_plus == opt_d3_plus

# Step 4: Minimize d4_plus (lowest priority goal)
prob.setObjective(d4_plus)
prob.solve()

# Print results
print("Status:", prob.status)
print("Gas Tax (G):", G.varValue)
print("Tax Rate on Income up to $30,000 (T1):", T1.varValue)
print("Tax Rate on Income over $30,000 (T2):", T2.varValue)
print("Spending Cut (C):", C.varValue)
print("Goal Deviations:")
print("d1_minus:", d1_minus.varValue, "d1_plus:", d1_plus.varValue)
print("d2_minus:", d2_plus.varValue, "d2_plus:", d1_plus.varValue)
print("d3_minus:", d3_plus.varValue, "d3_plus:", d1_plus.varValue)
print("d4_minus:", d4_minus.varValue, "d4_plus:", d4_plus.varValue)

Status: 1
Gas Tax (G): 300.0
Tax Rate on Income up to $30,000 (T1): 0.0
Tax Rate on Income over $30,000 (T2): 26.666667
Spending Cut (C): 150.0
Goal Deviations:
d1_minus: 0.0 d1_plus: 0.0
d2_minus: 0.0 d2_plus: 0.0
d3_minus: 0.0 d3_plus: 0.0
d4_minus: 50.0 d4_plus: 0.0


# **Question 3B**

In [ ]:
# Function to calculate total revenue
def calculate_total_revenue(G, T1, T2):
    revenue_low = G + 20 * T1
    revenue_high = 0.5 * G + 5 * T1 + 15 * T2
    return revenue_low + revenue_high

# Base values for G, T1, T2 from the solved problem
base_G = 300
base_T1 = 0
base_T2 = 26.666667

# Base revenue for comparison
base_revenue = calculate_total_revenue(base_G, base_T1, base_T2)

# Define percentage changes for sensitivity analysis
percentage_changes = [-0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3]

# Store results including revenue change
sensitivity_results = []

# Sensitivity for G
for change in percentage_changes:
    G = base_G * (1 + change)
    revenue = calculate_total_revenue(G, base_T1, base_T2)
    revenue_change = ((revenue - base_revenue) / base_revenue) * 100  # Revenue change in percentage
    sensitivity_results.append(("G", G, base_T1, base_T2, revenue, change * 100, revenue_change))

# Sensitivity for T1
for change in percentage_changes:
    T1 = base_T1 * (1 + change)
    revenue = calculate_total_revenue(base_G, T1, base_T2)
    revenue_change = ((revenue - base_revenue) / base_revenue) * 100  # Revenue change in percentage
    sensitivity_results.append(("T1", base_G, T1, base_T2, revenue, change * 100, revenue_change))

# Sensitivity for T2
for change in percentage_changes:
    T2 = base_T2 * (1 + change)
    revenue = calculate_total_revenue(base_G, base_T1, T2)
    revenue_change = ((revenue - base_revenue) / base_revenue) * 100  # Revenue change in percentage
    sensitivity_results.append(("T2", base_G, base_T1, T2, revenue, change * 100, revenue_change))

# Convert to DataFrame for analysis
sensitivity_df = pd.DataFrame(
    sensitivity_results,
    columns=["Variable", "G", "T1", "T2", "Revenue", "Percentage_Change", "Revenue_Change (%)"]
)

print(sensitivity_df)

   Variable      G   T1         T2     Revenue  Percentage_Change  \
0         G  210.0  0.0  26.666667  715.000005              -30.0   
1         G  240.0  0.0  26.666667  760.000005              -20.0   
2         G  270.0  0.0  26.666667  805.000005              -10.0   
3         G  300.0  0.0  26.666667  850.000005                0.0   
4         G  330.0  0.0  26.666667  895.000005               10.0   
5         G  360.0  0.0  26.666667  940.000005               20.0   
6         G  390.0  0.0  26.666667  985.000005               30.0   
7        T1  300.0  0.0  26.666667  850.000005              -30.0   
8        T1  300.0  0.0  26.666667  850.000005              -20.0   
9        T1  300.0  0.0  26.666667  850.000005              -10.0   
10       T1  300.0  0.0  26.666667  850.000005                0.0   
11       T1  300.0  0.0  26.666667  850.000005               10.0   
12       T1  300.0  0.0  26.666667  850.000005               20.0   
13       T1  300.0  0.0  26.666667

# **Question 4A**

In [ ]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum

# Data from the problem
cost_pc_to_wh = {
    ('A', 1): 4, ('A', 2): 3, ('A', 3): 5,
    ('B', 1): 6, ('B', 2): 2, ('B', 3): 4,
    ('C', 1): 5, ('C', 2): 4, ('C', 3): 3,
    ('D', 1): 7, ('D', 2): 5, ('D', 3): 6
}

cost_wh_to_dc = {
    (1, 1): 3, (1, 2): 2, (1, 3): 5, (1, 4): 4, (1, 5): 6, (1, 6): 7,
    (2, 1): 1, (2, 2): 4, (2, 3): 2, (2, 4): 3, (2, 5): 5, (2, 6): 6,
    (3, 1): 5, (3, 2): 3, (3, 3): 4, (3, 4): 2, (3, 5): 3, (3, 6): 4
}

supply_pc = {'A': 70, 'B': 90, 'C': 80, 'D': 60}
demand_dc = {1: 50, 2: 40, 3: 60, 4: 70, 5: 50, 6: 30}

# Warehouses balancing supply/demand
warehouse_indices = [1, 2, 3]
production_centres = ['A', 'B', 'C', 'D']
distribution_centres = [1, 2, 3, 4, 5, 6]

# Problem definition
problem = LpProblem("Supply_Chain_Optimization", LpMinimize)

# Decision Variables
x = LpVariable.dicts("PC_to_WH", cost_pc_to_wh.keys(), lowBound=0, cat='Continuous')
y = LpVariable.dicts("WH_to_DC", cost_wh_to_dc.keys(), lowBound=0, cat='Continuous')

# Objective Function
problem += lpSum(cost_pc_to_wh[i] * x[i] for i in cost_pc_to_wh) + lpSum(cost_wh_to_dc[j] * y[j] for j in cost_wh_to_dc), "Total_Cost"

# Constraints
# Supply constraints at production centres
for pc in production_centres:
    problem += lpSum(x[(pc, wh)] for wh in warehouse_indices) <= supply_pc[pc], f"Supply_Constraint_{pc}"

# Demand constraints at distribution centres
for dc in distribution_centres:
    problem += lpSum(y[(wh, dc)] for wh in warehouse_indices) >= demand_dc[dc], f"Demand_Constraint_{dc}"

# Balancing constraints at warehouses
for wh in warehouse_indices:
    problem += lpSum(x[(pc, wh)] for pc in production_centres) == lpSum(y[(wh, dc)] for dc in distribution_centres), f"Warehouse_Balance_{wh}"

# Solve the problem
problem.solve()

# Output results
print("Status:", problem.status)
print("Optimal Total Cost:", problem.objective.value())

print("\nOptimal Transport Plan (Production Centre to Warehouse):")
for i in cost_pc_to_wh:
    print(f"{i}: {x[i].value()}")

print("\nOptimal Transport Plan (Warehouse to Distribution Centre):")
for j in cost_wh_to_dc:
    print(f"{j}: {y[j].value()}")


Status: 1
Optimal Total Cost: 1700.0

Optimal Transport Plan (Production Centre to Warehouse):
('A', 1): 40.0
('A', 2): 30.0
('A', 3): 0.0
('B', 1): 0.0
('B', 2): 90.0
('B', 3): 0.0
('C', 1): 0.0
('C', 2): 0.0
('C', 3): 80.0
('D', 1): 0.0
('D', 2): 60.0
('D', 3): 0.0

Optimal Transport Plan (Warehouse to Distribution Centre):
(1, 1): 0.0
(1, 2): 40.0
(1, 3): 0.0
(1, 4): 0.0
(1, 5): 0.0
(1, 6): 0.0
(2, 1): 50.0
(2, 2): 0.0
(2, 3): 60.0
(2, 4): 70.0
(2, 5): 0.0
(2, 6): 0.0
(3, 1): 0.0
(3, 2): 0.0
(3, 3): 0.0
(3, 4): 0.0
(3, 5): 50.0
(3, 6): 30.0


# **Question** **4B**

In [ ]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum

# Data from the problem (updated costs for Warehouse 2)
cost_pc_to_wh = {
    ('A', 1): 4, ('A', 2): 4, ('A', 3): 5,
    ('B', 1): 6, ('B', 2): 3, ('B', 3): 4,
    ('C', 1): 5, ('C', 2): 5, ('C', 3): 3,
    ('D', 1): 7, ('D', 2): 6, ('D', 3): 6
}

cost_wh_to_dc = {
    (1, 1): 3, (1, 2): 2, (1, 3): 5, (1, 4): 4, (1, 5): 6, (1, 6): 7,
    (2, 1): 2, (2, 2): 5, (2, 3): 3, (2, 4): 4, (2, 5): 6, (2, 6): 7,
    (3, 1): 5, (3, 2): 3, (3, 3): 4, (3, 4): 2, (3, 5): 3, (3, 6): 4
}

supply_pc = {'A': 70, 'B': 90, 'C': 80, 'D': 60}
demand_dc = {1: 50, 2: 40, 3: 60, 4: 70, 5: 50, 6: 30}

# Warehouses balancing supply/demand
warehouse_indices = [1, 2, 3]
production_centres = ['A', 'B', 'C', 'D']
distribution_centres = [1, 2, 3, 4, 5, 6]

# Problem definition
problem = LpProblem("Supply_Chain_Optimization", LpMinimize)

# Decision Variables
x = LpVariable.dicts("PC_to_WH", cost_pc_to_wh.keys(), lowBound=0, cat='Continuous')
y = LpVariable.dicts("WH_to_DC", cost_wh_to_dc.keys(), lowBound=0, cat='Continuous')

# Objective Function
problem += lpSum(cost_pc_to_wh[i] * x[i] for i in cost_pc_to_wh) + lpSum(cost_wh_to_dc[j] * y[j] for j in cost_wh_to_dc), "Total_Cost"

# Constraints
# Supply constraints at production centres
for pc in production_centres:
    problem += lpSum(x[(pc, wh)] for wh in warehouse_indices) <= supply_pc[pc], f"Supply_Constraint_{pc}"

# Demand constraints at distribution centres
for dc in distribution_centres:
    problem += lpSum(y[(wh, dc)] for wh in warehouse_indices) >= demand_dc[dc], f"Demand_Constraint_{dc}"

# Balancing constraints at warehouses
for wh in warehouse_indices:
    problem += lpSum(x[(pc, wh)] for pc in production_centres) == lpSum(y[(wh, dc)] for dc in distribution_centres), f"Warehouse_Balance_{wh}"

# Solve the problem
problem.solve()

# Output results
print("Status:", problem.status)
print("Optimal Total Cost:", problem.objective.value())

print("\nOptimal Transport Plan (Production Centre to Warehouse):")
for i in cost_pc_to_wh:
    print(f"{i}: {x[i].value()}")

print("\nOptimal Transport Plan (Warehouse to Distribution Centre):")
for j in cost_wh_to_dc:
    print(f"{j}: {y[j].value()}")


Status: 1
Optimal Total Cost: 1930.0

Optimal Transport Plan (Production Centre to Warehouse):
('A', 1): 40.0
('A', 2): 30.0
('A', 3): 0.0
('B', 1): 0.0
('B', 2): 80.0
('B', 3): 10.0
('C', 1): 0.0
('C', 2): 0.0
('C', 3): 80.0
('D', 1): 0.0
('D', 2): 0.0
('D', 3): 60.0

Optimal Transport Plan (Warehouse to Distribution Centre):
(1, 1): 0.0
(1, 2): 40.0
(1, 3): 0.0
(1, 4): 0.0
(1, 5): 0.0
(1, 6): 0.0
(2, 1): 50.0
(2, 2): 0.0
(2, 3): 60.0
(2, 4): 0.0
(2, 5): 0.0
(2, 6): 0.0
(3, 1): 0.0
(3, 2): 0.0
(3, 3): 0.0
(3, 4): 70.0
(3, 5): 50.0
(3, 6): 30.0


# **Question 4C**

In [ ]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum

# Data from the problem
cost_pc_to_wh = {
    ('A', 1): 4, ('A', 2): 3, ('A', 3): 5,
    ('B', 1): 6, ('B', 2): 2, ('B', 3): 4,
    ('C', 1): 5, ('C', 2): 4, ('C', 3): 3,
    ('D', 1): 7, ('D', 2): 5, ('D', 3): 6
}

cost_wh_to_dc = {
    (1, 1): 3, (1, 2): 2, (1, 3): 5, (1, 4): 4, (1, 5): 6, (1, 6): 7,
    (2, 1): 1, (2, 2): 4, (2, 3): 2, (2, 4): 3, (2, 5): 5, (2, 6): 6,
    (3, 1): 5, (3, 2): 3, (3, 3): 4, (3, 4): 2, (3, 5): 3, (3, 6): 4
}

supply_pc = {'A': 70, 'B': 90, 'C': 80, 'D': 60}
demand_dc = {1: 50, 2: 40, 3: 60, 4: 70, 5: 50, 6: 30}

# Warehouses balancing supply/demand
warehouse_indices = [1, 2, 3]
production_centres = ['A', 'B', 'C', 'D']
distribution_centres = [1, 2, 3, 4, 5, 6]

# Problem definition
problem = LpProblem("Supply_Chain_Optimization", LpMinimize)

# Decision Variables
x = LpVariable.dicts("PC_to_WH", cost_pc_to_wh.keys(), lowBound=0, cat='Continuous')
y = LpVariable.dicts("WH_to_DC", cost_wh_to_dc.keys(), lowBound=0, cat='Continuous')

# Objective Function
problem += lpSum(cost_pc_to_wh[i] * x[i] for i in cost_pc_to_wh) + lpSum(cost_wh_to_dc[j] * y[j] for j in cost_wh_to_dc), "Total_Cost"

# Constraints
# Supply constraints at production centres
for pc in production_centres:
    problem += lpSum(x[(pc, wh)] for wh in warehouse_indices) <= supply_pc[pc], f"Supply_Constraint_{pc}"

# Demand constraints at distribution centres
for dc in distribution_centres:
    problem += lpSum(y[(wh, dc)] for wh in warehouse_indices) >= demand_dc[dc], f"Demand_Constraint_{dc}"

# Balancing constraints at warehouses
for wh in warehouse_indices:
    problem += lpSum(x[(pc, wh)] for pc in production_centres) == lpSum(y[(wh, dc)] for dc in distribution_centres), f"Warehouse_Balance_{wh}"

# Handling capacity constraint for Warehouse 1
problem += (
    lpSum(x[(pc, 1)] for pc in production_centres) <= 50,
    "Handling_Capacity_Warehouse_1"
)

# Solve the problem
problem.solve()

# Output results
print("Status:", problem.status)
print("Optimal Total Cost:", problem.objective.value())

print("\nOptimal Transport Plan (Production Centre to Warehouse):")
for i in cost_pc_to_wh:
    print(f"{i}: {x[i].value()}")

print("\nOptimal Transport Plan (Warehouse to Distribution Centre):")
for j in cost_wh_to_dc:
    print(f"{j}: {y[j].value()}")

Status: 1
Optimal Total Cost: 1700.0

Optimal Transport Plan (Production Centre to Warehouse):
('A', 1): 40.0
('A', 2): 30.0
('A', 3): 0.0
('B', 1): 0.0
('B', 2): 90.0
('B', 3): 0.0
('C', 1): 0.0
('C', 2): 0.0
('C', 3): 80.0
('D', 1): 0.0
('D', 2): 60.0
('D', 3): 0.0

Optimal Transport Plan (Warehouse to Distribution Centre):
(1, 1): 0.0
(1, 2): 40.0
(1, 3): 0.0
(1, 4): 0.0
(1, 5): 0.0
(1, 6): 0.0
(2, 1): 50.0
(2, 2): 0.0
(2, 3): 60.0
(2, 4): 70.0
(2, 5): 0.0
(2, 6): 0.0
(3, 1): 0.0
(3, 2): 0.0
(3, 3): 0.0
(3, 4): 0.0
(3, 5): 50.0
(3, 6): 30.0


# **Question 4D**

In [ ]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum

# Data from the problem
cost_pc_to_wh = {
    ('A', 1): 4, ('A', 2): 3, ('A', 3): 5,
    ('B', 1): 6, ('B', 2): 2, ('B', 3): 4,
    ('C', 1): 5, ('C', 2): 4, ('C', 3): 3,
    ('D', 1): 7, ('D', 2): 5, ('D', 3): 6
}

cost_wh_to_dc = {
    (1, 1): 3, (1, 2): 2, (1, 3): 5, (1, 4): 4, (1, 5): 6, (1, 6): 7,
    (2, 1): 1, (2, 2): 4, (2, 3): 2, (2, 4): 3, (2, 5): 5, (2, 6): 6,
    (3, 1): 5, (3, 2): 3, (3, 3): 4, (3, 4): 2, (3, 5): 3, (3, 6): 4
}

supply_pc = {'A': 70, 'B': 90, 'C': 80, 'D': 60}
demand_dc = {1: 50, 2: 40, 3: 60, 4: 70, 5: 50, 6: 30}

# Warehouses balancing supply/demand
warehouse_indices = [1, 2, 3]
production_centres = ['A', 'B', 'C', 'D']
distribution_centres = [1, 2, 3, 4, 5, 6]

def solve_problem(demand_dc, removed_dc):
    # Problem definition
    problem = LpProblem("Supply_Chain_Optimization", LpMinimize)

    # Decision Variables
    x = LpVariable.dicts("PC_to_WH", cost_pc_to_wh.keys(), lowBound=0, cat='Continuous')
    y = LpVariable.dicts("WH_to_DC", cost_wh_to_dc.keys(), lowBound=0, cat='Continuous')

    # Objective Function
    problem += lpSum(cost_pc_to_wh[i] * x[i] for i in cost_pc_to_wh) + lpSum(cost_wh_to_dc[j] * y[j] for j in cost_wh_to_dc), "Total_Cost"

    # Constraints
    # Supply constraints at production centres
    for pc in production_centres:
        problem += lpSum(x[(pc, wh)] for wh in warehouse_indices) <= supply_pc[pc], f"Supply_Constraint_{pc}"

    # Demand constraints at distribution centres
    for dc in distribution_centres:
        if dc != removed_dc:
            problem += lpSum(y[(wh, dc)] for wh in warehouse_indices) >= demand_dc[dc], f"Demand_Constraint_{dc}"

    # Balancing constraints at warehouses
    for wh in warehouse_indices:
        problem += lpSum(x[(pc, wh)] for pc in production_centres) == lpSum(y[(wh, dc)] for dc in distribution_centres if dc != removed_dc), f"Warehouse_Balance_{wh}"

    # Solve the problem
    problem.solve()

    # Output results
    print(f"\nResults after removing DC{removed_dc}:")
    print("Status:", problem.status)
    print("Optimal Total Cost:", problem.objective.value())

    print("\nOptimal Transport Plan (Production Centre to Warehouse):")
    for i in cost_pc_to_wh:
        print(f"{i}: {x[i].value()}")

    print("\nOptimal Transport Plan (Warehouse to Distribution Centre):")
    for j in cost_wh_to_dc:
        if j[1] != removed_dc:
            print(f"{j}: {y[j].value()}")

# Remove DC1
removed_dc1 = 1
redistributed_demand_dc1 = demand_dc.copy()
demand_to_redistribute1 = demand_dc[removed_dc1] / (len(distribution_centres) - 1)
for dc in distribution_centres:
    if dc != removed_dc1:
        redistributed_demand_dc1[dc] += demand_to_redistribute1
redistributed_demand_dc1[removed_dc1] = 0
solve_problem(redistributed_demand_dc1, removed_dc1)

# Remove DC5
removed_dc5 = 5
redistributed_demand_dc5 = demand_dc.copy()
demand_to_redistribute5 = demand_dc[removed_dc5] / (len(distribution_centres) - 1)
for dc in distribution_centres:
    if dc != removed_dc5:
        redistributed_demand_dc5[dc] += demand_to_redistribute5
redistributed_demand_dc5[removed_dc5] = 0
solve_problem(redistributed_demand_dc5, removed_dc5)



Results after removing DC1:
Status: 1
Optimal Total Cost: 1820.0

Optimal Transport Plan (Production Centre to Warehouse):
('A', 1): 50.0
('A', 2): 20.0
('A', 3): 0.0
('B', 1): 0.0
('B', 2): 90.0
('B', 3): 0.0
('C', 1): 0.0
('C', 2): 0.0
('C', 3): 80.0
('D', 1): 0.0
('D', 2): 0.0
('D', 3): 60.0

Optimal Transport Plan (Warehouse to Distribution Centre):
(1, 2): 50.0
(1, 3): 0.0
(1, 4): 0.0
(1, 5): 0.0
(1, 6): 0.0
(2, 2): 0.0
(2, 3): 70.0
(2, 4): 40.0
(2, 5): 0.0
(2, 6): 0.0
(3, 2): 0.0
(3, 3): 0.0
(3, 4): 40.0
(3, 5): 60.0
(3, 6): 40.0

Results after removing DC5:
Status: 1
Optimal Total Cost: 1640.0

Optimal Transport Plan (Production Centre to Warehouse):
('A', 1): 50.0
('A', 2): 20.0
('A', 3): 0.0
('B', 1): 0.0
('B', 2): 90.0
('B', 3): 0.0
('C', 1): 0.0
('C', 2): 0.0
('C', 3): 80.0
('D', 1): 0.0
('D', 2): 60.0
('D', 3): 0.0

Optimal Transport Plan (Warehouse to Distribution Centre):
(1, 1): 0.0
(1, 2): 50.0
(1, 3): 0.0
(1, 4): 0.0
(1, 6): 0.0
(2, 1): 60.0
(2, 2): 0.0
(2, 3): 70.0
(

In [ ]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum

# Data from the problem
cost_pc_to_wh = {
    ('A', 1): 4, ('A', 2): 3, ('A', 3): 5,
    ('B', 1): 6, ('B', 2): 2, ('B', 3): 4,
    ('C', 1): 5, ('C', 2): 4, ('C', 3): 3,
    ('D', 1): 7, ('D', 2): 5, ('D', 3): 6
}

cost_wh_to_dc = {
    (1, 1): 3, (1, 2): 2, (1, 3): 5, (1, 4): 4, (1, 5): 6, (1, 6): 7,
    (2, 1): 1, (2, 2): 4, (2, 3): 2, (2, 4): 3, (2, 5): 5, (2, 6): 6,
    (3, 1): 5, (3, 2): 3, (3, 3): 4, (3, 4): 2, (3, 5): 3, (3, 6): 4
}

supply_pc = {'A': 70, 'B': 90, 'C': 80, 'D': 60}
demand_dc = {1: 50, 2: 40, 3: 60, 4: 70, 5: 50, 6: 30}

# Warehouses balancing supply/demand
warehouse_indices = [1, 2, 3]
production_centres = ['A', 'B', 'C', 'D']
distribution_centres = [1, 2, 3, 4, 5, 6]

def solve_problem(demand_dc, removed_dc):
    # Define the problem
    problem = LpProblem("Supply_Chain_Optimization", LpMinimize)

    # Define decision variables
    x = LpVariable.dicts("PC_to_WH", cost_pc_to_wh.keys(), lowBound=0, cat='Continuous')
    y = LpVariable.dicts("WH_to_DC", cost_wh_to_dc.keys(), lowBound=0, cat='Continuous')

    # Define the objective function
    problem += (
        lpSum(cost_pc_to_wh[i] * x[i] for i in cost_pc_to_wh) +
        lpSum(cost_wh_to_dc[j] * y[j] for j in cost_wh_to_dc)
    ), "Total_Transportation_Cost"

    # Supply constraints for production centres
    for pc in production_centres:
        problem += lpSum(x[(pc, wh)] for wh in warehouse_indices) <= supply_pc[pc], f"Supply_Constraint_{pc}"

    # Demand constraints for distribution centres (excluding removed DC)
    for dc in distribution_centres:
        if dc != removed_dc:
            problem += lpSum(y[(wh, dc)] for wh in warehouse_indices) >= demand_dc[dc], f"Demand_Constraint_{dc}"

    # Balancing constraints at warehouses
    for wh in warehouse_indices:
        problem += (
            lpSum(x[(pc, wh)] for pc in production_centres) ==
            lpSum(y[(wh, dc)] for dc in distribution_centres if dc != removed_dc),
            f"Warehouse_Balance_{wh}"
        )

    # Solve the problem
    problem.solve()

    # Output results
    print(f"\nResults after removing DC{removed_dc}:")
    print("Status:", problem.status)
    print("Optimal Total Cost:", problem.objective.value())

    print("\nOptimal Transport Plan (Production Centre to Warehouse):")
    for i in cost_pc_to_wh:
        print(f"{i}: {x[i].value()}")

    print("\nOptimal Transport Plan (Warehouse to Distribution Centre):")
    for j in cost_wh_to_dc:
        if j[1] != removed_dc:
            print(f"{j}: {y[j].value()}")

# Function to redistribute demand when a distribution centre is removed
def redistribute_demand(demand_dc, removed_dc):
    redistributed_amount = demand_dc[removed_dc] / (len(distribution_centres) - 1)
    return {dc: (demand_dc[dc] + redistributed_amount if dc != removed_dc else 0) for dc in distribution_centres}

# Scenario: Remove DC1
print("Scenario: Remove DC1")
redistributed_demand_dc1 = redistribute_demand(demand_dc, 1)
solve_problem(redistributed_demand_dc1, 1)

# Scenario: Remove DC5
print("\nScenario: Remove DC5")
redistributed_demand_dc5 = redistribute_demand(demand_dc, 5)
solve_problem(redistributed_demand_dc5, 5)

Scenario: Remove DC1

Results after removing DC1:
Status: 1
Optimal Total Cost: 1820.0

Optimal Transport Plan (Production Centre to Warehouse):
('A', 1): 50.0
('A', 2): 20.0
('A', 3): 0.0
('B', 1): 0.0
('B', 2): 90.0
('B', 3): 0.0
('C', 1): 0.0
('C', 2): 0.0
('C', 3): 80.0
('D', 1): 0.0
('D', 2): 0.0
('D', 3): 60.0

Optimal Transport Plan (Warehouse to Distribution Centre):
(1, 2): 50.0
(1, 3): 0.0
(1, 4): 0.0
(1, 5): 0.0
(1, 6): 0.0
(2, 2): 0.0
(2, 3): 70.0
(2, 4): 40.0
(2, 5): 0.0
(2, 6): 0.0
(3, 2): 0.0
(3, 3): 0.0
(3, 4): 40.0
(3, 5): 60.0
(3, 6): 40.0

Scenario: Remove DC5

Results after removing DC5:
Status: 1
Optimal Total Cost: 1640.0

Optimal Transport Plan (Production Centre to Warehouse):
('A', 1): 50.0
('A', 2): 20.0
('A', 3): 0.0
('B', 1): 0.0
('B', 2): 90.0
('B', 3): 0.0
('C', 1): 0.0
('C', 2): 0.0
('C', 3): 80.0
('D', 1): 0.0
('D', 2): 60.0
('D', 3): 0.0

Optimal Transport Plan (Warehouse to Distribution Centre):
(1, 1): 0.0
(1, 2): 50.0
(1, 3): 0.0
(1, 4): 0.0
(1, 6): 